<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
display(data.head())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
display(data[['Возраст', 'Зарплата', 'Члены семьи']].describe().loc[['min', 'max']])

,Возраст,Зарплата,Члены семьи
min,18.0,5300.0,0.0
max,65.0,79000.0,6.0


    0 - членов семьи, скорее всего, значит, что семьи нет.
    Считаю, что в данных нет аномальных значений

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

# Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится

**Обоснование:** Так как пресказания вычиляюся по формуле

$$
a = X\cdot w
$$

Тогда после умножения признаков на обратимую матрицу М получаем:

$$
a_{1} = (M\cdot X_{1})w
$$

или
$$
a_{1} = (M\cdot X_{1})((M\cdot X)^T\cdot M\cdot X)^{-1} (M\cdot X)^T\cdot y
$$


$$
a_{1} = (M\cdot X_{1})\cdot(M^T)^{-1}\cdot (X^T\cdot X)^{-1}\cdot M^{-1}\cdot (X^T\cdot M^T)\cdot y
$$


$$
a_{1} = E\cdot X_{1}\cdot E \cdot (X^T\cdot X)^{-1}\cdot X^T\cdot y
$$

при умножении на единичную матрицу (справа или слева) исходная матрица не изменяется.

$$
a_{1} =  X_{1}\cdot(X^T\cdot X)^{-1} \cdot X^T\cdot y
$$


$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} E X^T y
$$

## Алгоритм преобразования

**Алгоритм**
$\cdot $ Оцениваю качество линейной регресси 

$\cdot $ Cоздаю случайную матрицу количество строк которой равно количеству столбцов обучающей выборки, также матрица должна быть квадратной

$\cdot $ Проверяю, что матрица обратимая

$\cdot $ Умножаю обучающую выборку на матрицу

$\cdot $ Вновь провожу оценку качества

**Обоснование**
$\cdot $ Нужна первоначальная оценка линейной регрессии, для того, чтобы было с чем сравнивать

$\cdot $ Умножение матрицы на матрицу возможно, если ширина первой матрицы $ А (𝑚\times 𝑛) $ равна высоте второй матрицы $ В (𝑛 \times r) $. 

$\cdot $ Матрица скорее всего обратима, но нужно проверить

$\cdot $ По двум оценкам можно проверить как изменились предсказания


## Проверка алгоритма

Разделил данные на целевой признак и обучающую выборку

In [5]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

Создал класс модели обучения с атрибутами обучения и предсказания

In [6]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        #print(self.w, self.w0, sep='\n')

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

Создал экземпляр класса, обучил, рассчитал и оценил предсказания

In [7]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
rate_a = r2_score(target, predictions)
display(rate_a)

0.42494550286668

Создал случаную матрицу, обратил её, умножил на неё обучающую выборку

In [8]:
a = np.random.randint(10, size = (4, 4)) # случайная матрица
a_inv = np.linalg.inv(a) # убедиться, что умножаю на обратимую матрицу
features_inv = features.dot(a_inv)

Повторил обучение, рассчет предсказаний и оценку качества предсазаний

In [9]:
model.fit(features_inv, target)
predictions = model.predict(features_inv)
rate_a_inv = r2_score(target, predictions)
display(rate_a_inv)

0.4249455028651892

In [10]:
display(rate_a/rate_a_inv)

1.0000000000035083

In [11]:
display(features_inv.head())

,0,1,2,3
0,-8069.037886,21520.444666,-8118.068794,-4990.121635
1,-6185.081755,16491.064806,-6222.622134,-3821.578265
2,-3418.920239,9114.327019,-3439.539382,-2111.533400
3,-6780.908275,18089.976072,-6822.570289,-4196.863410
4,-4247.394816,11325.581256,-4272.980060,-2625.109671


In [12]:
display(features_inv.dot(a).head().round(0))

,0,1,2,3
0,1.0,41.0,49600.0,1.0
1,0.0,46.0,38000.0,1.0
2,0.0,29.0,21000.0,0.0
3,0.0,21.0,41700.0,2.0
4,1.0,28.0,26100.0,0.0


    Оценки качества линейной регресии совпадает почти на 100 %, это значит, что умножение признаков на обратимую матрицу никак не повлияли на качество регресии.
    При этом сами данные понять невозможно, до тех пор, пока не произведено умножение полученных признаков на матрицу, обратную той, на которую умножалось в первый раз.